In [ ]:
!pip install requests bs4 pandas numpy datetime urllib3 tqdm reachability rpy2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
url = "https://www.courtlistener.com/api/rest/v3/"
api_key = "163cea228fb27936988d579ed72fe787848e1866"


In [ ]:
import pickle
import requests
from urllib.parse import urlencode
from tqdm import tqdm
import time
from bs4 import BeautifulSoup as beautifulsoup
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
from reachability import Reachability
import os
import multiprocessing

def get_data_from_sub_opinions(api_key, url, sub_opinions_ids, max_queries_per_hour, query_count, query_lock):
    url = url + "opinions/"
    headers = {"Authorization": f"Token {api_key}"}
    opinions_data = []

    start_time = time.time()
    with tqdm(total=len(sub_opinions_ids), desc="Fetching sub-opinions data", unit="page") as pbar:
        seq_index = 0
        while seq_index < len(sub_opinions_ids):
            seq = sub_opinions_ids[seq_index]
            params = {
                "id__gte": seq[0],
                "id__lte": seq[1],
                "author__gte": 0,
                "order_by": "id",
                "fields": "plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id",
            }
            while True:
                try:
                    # Check if the rate limit has been reached
                    with query_lock:
                        if query_count.value >= max_queries_per_hour:
                            elapsed_time = time.time() - start_time
                            if elapsed_time < 3600:  # 3600 seconds = 1 hour
                                sleep_time = 3600 - elapsed_time
                                print(f"Rate limit reached. Waiting for {sleep_time:.2f} seconds...")
                                time.sleep(sleep_time)
                                start_time = time.time()
                                query_count.value = 0

                    newurl = f"{url}?{urlencode(params, safe=',')}"
                    response = requests.get(newurl, headers=headers)
                    response.raise_for_status()
                    data = response.json()
                    opinions_data.extend(data["results"])

                    if not data["next"]:
                        break
                    params["id__gte"] = data["results"][-1]["id"] + 1

                    with query_lock:
                        query_count.value += 1
                    time.sleep(1)
                except requests.exceptions.HTTPError as err:
                    # If the request fails, wait 3 minutes and try again with the same parameters
                    print(err)
                    time.sleep(180)
                    continue
                except requests.exceptions.ConnectionError as err:
                    reachable = Reachability(timeout=None)
                    while not reachable.is_online():
                        print("Waiting for internet connection...")
                        time.sleep(120)
                    continue
                except Exception as err:
                    print("Other error: ", err)
                    break

            pbar.update(1)
            seq_index += 1

    return opinions_data

def worker(api_key, url, sub_opinions_ids, max_queries_per_hour, query_count, query_lock, worker_id):
    opinions_data = get_data_from_sub_opinions(api_key, url, sub_opinions_ids, max_queries_per_hour, query_count, query_lock)
    with open(f'/content/drive/MyDrive/opinions_data_{worker_id}.pickle', 'wb') as f:
        pickle.dump(opinions_data, f)
    print(f"Worker {worker_id} - Fetched opinions data saved as 'opinions_data_{worker_id}.pickle'")

if __name__ == '__main__':
    from google.colab import drive
    drive.mount('/content/drive')

    # Check if remaining_seq.pickle exists
    if os.path.exists('/content/drive/MyDrive/remaining_seq.pickle'):
        with open('/content/drive/MyDrive/remaining_seq.pickle', 'rb') as f:
            remaining_sequences = pickle.load(f)
    else:
        print("remaining_seq.pickle not found. Please ensure the file exists.")
        sys.exit(1)

    # Set up multiprocessing
    num_workers = 4  # Specify the number of worker processes
    chunk_size = len(remaining_sequences) // num_workers

    max_queries_per_hour = 5000
    query_count = multiprocessing.Value('i', 0)  # Shared variable for total query count
    query_lock = multiprocessing.Lock()  # Lock for synchronizing access to query_count

    workers = []
    for i in range(num_workers):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size if i < num_workers - 1 else len(remaining_sequences)
        sub_opinions_ids_chunk = remaining_sequences[start_index:end_index]
        worker_process = multiprocessing.Process(target=worker, args=(api_key, url, sub_opinions_ids_chunk, max_queries_per_hour, query_count, query_lock, i))
        workers.append(worker_process)
        worker_process.start()

    for worker_process in workers:
        worker_process.join()

    print("All workers completed.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fetching sub-opinions data:   8%|▊         | 257/3154 [01:27<12:18,  3.92page/s]

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=6353242&id__lte=6353243&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id


Fetching sub-opinions data:  10%|█         | 322/3154 [01:42<12:36,  3.75page/s]

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=8331147&id__lte=8331147&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id


...

502 Server Error: Bad Gateway for url: https://www.courtlistener.com/api/rest/v3/opinions/?id__gte=4412107&id__lte=4412107&author__gte=0&order_by=id&fields=plain_text,html,html_lawbox,html_columbia,html_anon_2020,xml_harvard,html_with_citations,opinions_cited,author_id,id,cluster_id


Fetching sub-opinions data:  78%|███████▊  | 2466/3154 [2:11:32<02:54,  3.95page/s]

Worker 3 - Fetched opinions data saved as 'opinions_data_3.pickle'


Fetching sub-opinions data: 100%|██████████| 3154/3154 [2:15:17<00:00,  2.57s/page]


Worker 0 - Fetched opinions data saved as 'opinions_data_0.pickle'
All workers completed.
